In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras import optimizers
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

Using TensorFlow backend.


In [3]:
df = pd.read_csv('Amreli NRM data for model building final.csv')
campaign = np.unique(df['campaign'])

for c in campaign:
    dfc = df[df['campaign']==c]
    i = dfc.index[0]
    df = df.drop(i)
    
df = df.reset_index(drop=True)
df['Number of capacitors removed'].fillna(0, inplace=True)

threshold = 150

for i in range(df.shape[0]):
    if(df.loc[i,'Total Heat Time previous (minutes)']>threshold):
        df.loc[i,'Total Heat Time previous (minutes)']= threshold
    if(df.loc[i,'Total Heat Time (minutes)']>threshold):
        df.loc[i,'Total Heat Time (minutes)']= threshold
        
df['Input energy previous (KWH)'] = df['Power previous']*df['Total Heat Time previous (minutes)']/60
df['Input energy (KWH)'] = df['Power']*df['Total Heat Time (minutes)']/60

In [4]:
df.columns

Index(['campaign', 'Heat number (life number)',
       'Total Heat Time previous (minutes)', 'Total Heat Time (minutes)',
       'Scrap (MT)', 'Sponge (MT)', 'Hot Heel (MT)', 'Alloys (MT)',
       'Input (MT)', 'Final C%', 'Tapping Temperature (C)', 'Output (MT)',
       'Input energy previous (KWH)', 'Input energy (KWH)', 'Previous Voltage',
       'Previous Frequency', 'Previous Rating of the Furnace', 'Voltage',
       'Frequency', 'Rating of the furnace (KW)', 'Melt rate (MT/Hr)',
       'Efficiency (MT/KWH)', 'Number of capacitors removed',
       'Refractory lining before heating', 'Refractory lining after heating',
       'Previous Input', 'Previous Output', 'Power', 'Power previous'],
      dtype='object')

In [5]:
df1 = df[df['Number of capacitors removed']==0].reset_index(drop=True)
df2 = df[df['Number of capacitors removed']!=0].reset_index(drop=True)

# Without Capacitor removal

In [12]:
input_columns = ['Scrap (MT)', 'Sponge (MT)', 'Hot Heel (MT)', 'Alloys (MT)','Previous Voltage','Previous Frequency','Refractory lining before heating','Number of capacitors removed','Tapping Temperature (C)']
output_columns = ['Voltage','Frequency','Refractory lining after heating']

X_s = df1[input_columns]
from sklearn.preprocessing import MinMaxScaler
scaler1 = MinMaxScaler()
scaler2 = MinMaxScaler()
X = scaler1.fit_transform(X_s)

X_train = X[:191,:]
X_test = X[191:,:]
y_s = df1[output_columns]
y_s1 = y_s.iloc[:191,:].values
y_s2 = y_s.iloc[191:,:].values
y = scaler2.fit_transform(y_s)
y_train = y[:191,:]
y_test = y[191:,:]

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_train1 = lr.predict(X_train)
y_pred_test1 = lr.predict(X_test)

y_pred_train = scaler2.inverse_transform(y_pred_train1)
y_pred_test = scaler2.inverse_transform(y_pred_test1)


for i in range(3):
    print(output_columns[i])
    print("Train score is {}".format(mean_squared_error(y_s1[:,i], y_pred_train[:,i])))
    print("Test score is {}".format(mean_squared_error(y_s2[:,i], y_pred_test[:,i])))
    print("R2 score is {}".format(r2_score(y_s2[:,i], y_pred_test[:,i])))
    print("Train NMSE score is {}".format(np.sum(np.square(y_pred_train[:,i]-y_s1[:,i]))/np.sum(np.square(y_s1[:,i]-np.mean(y_s1[:,i])))*100))
    print("Test NMSE score is {}".format(np.sum(np.square(y_pred_test[:,i]-y_s2[:,i]))/np.sum(np.square(y_s2[:,i]-np.mean(y_s2[:,i])))*100))

for i in range(lr.coef_.shape[0]):
    for j in range(lr.coef_.shape[1]):
        print("Weight of {} on {} is {}".format(input_columns[j], output_columns[i],lr.coef_[i][j]))
    print('\n')

Voltage
Train score is 2573.0622978765145
Test score is 1544.498060086697
R2 score is 0.9355223393507034
Train NMSE score is 15.097670244450626
Test NMSE score is 6.447766064929665
Frequency
Train score is 9.199292614815265
Test score is 2.886873758281543
R2 score is 0.935901742398932
Train NMSE score is 10.607350751874012
Test NMSE score is 6.409825760106805
Refractory lining after heating
Train score is 2.7031194211736075e-05
Test score is 7.175876376530534e-05
R2 score is 0.9997666695556623
Train NMSE score is 0.008147182157644605
Test NMSE score is 0.023333044433776
Weight of Scrap (MT) on Voltage is -0.002867865180835507
Weight of Sponge (MT) on Voltage is 0.0017990143996006577
Weight of Hot Heel (MT) on Voltage is 2.220446049250313e-16
Weight of Alloys (MT) on Voltage is -0.048026227907818866
Weight of Previous Voltage on Voltage is 0.8380754202749178
Weight of Previous Frequency on Voltage is -0.19007486341281804
Weight of Refractory lining before heating on Voltage is 0.1072893

In [14]:
import pickle
pickle.dump(lr, open('lin_model_no_capacitors_removed.pkl', 'wb'))
pickle.dump(scaler1, open('scaler_input_no_capacitors_removed.pkl', 'wb'))
pickle.dump(scaler2, open('scaler_output_no_capacitors_removed.pkl', 'wb'))

# Capacitors removed

In [6]:
input_columns = ['Scrap (MT)', 'Sponge (MT)', 'Hot Heel (MT)', 'Alloys (MT)','Previous Voltage','Previous Frequency','Refractory lining before heating','Number of capacitors removed','Tapping Temperature (C)']
output_columns = ['Voltage','Frequency','Refractory lining after heating']

X = df2[input_columns]
X_train = X.iloc[:127,:].values
X_test = X.iloc[127:,:].values
y = df2[output_columns]
y_train = y.iloc[:127,:].values
y_test = y.iloc[127:,:].values

scaler1 = MinMaxScaler()
normalized_X_train = scaler1.fit_transform(X_train)
normalized_X_test = scaler1.transform(X_test)

scaler2 = MinMaxScaler()
normalized_y_train = scaler2.fit_transform(y_train)
normalized_y_test = scaler2.transform(y_test)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
n_hidden_1 = 500
n_hidden_2 = 500
n_hidden_3 = 500
n_input = X_train.shape[1] 
n_output = 3
model = Sequential()
model.add(Dense(n_hidden_1, input_dim=n_input, activation='relu'))
model.add(Dense(n_hidden_2, activation='relu'))
model.add(Dense(n_hidden_3, activation='relu'))
model.add(Dense(n_output, activation='relu'))
# compile the keras model
model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer='adam', metrics=['mse'])
# fit the keras model on the dataset
model.fit(normalized_X_train, normalized_y_train, epochs=500, batch_size=5, verbose=2)
# make class predictions with the model

pred_train = model.predict(normalized_X_train)
y_pred_train = scaler2.inverse_transform(pred_train)
pred_test = model.predict(normalized_X_test)
y_pred_test = scaler2.inverse_transform(pred_test)

num_digits=3
for i in range(num_digits):
    print(output_columns[i])
    print("Train score is {}".format(mean_squared_error(y_train[:,i], y_pred_train[:,i])))
    print("Test score is {}".format(mean_squared_error(y_test[:,i], y_pred_test[:,i])))
    print("R2 score is {}".format(r2_score(y_test[:,i], y_pred_test[:,i])))
    print("Train NMSE score is {}".format(np.sum(np.square(y_pred_train[:,i]-y_train[:,i]))/np.sum(np.square(y_train[:,i]-np.mean(y_train[:,i])))*100))
    print("Test NMSE score is {}".format(np.sum(np.square(y_pred_test[:,i]-y_test[:,i]))/np.sum(np.square(y_test[:,i]-np.mean(y_test[:,i])))*100))

Train on 127 samples
Epoch 1/500
127/127 - 1s - loss: 0.0423 - mse: 0.0423
Epoch 2/500
127/127 - 0s - loss: 0.0148 - mse: 0.0148
Epoch 3/500
127/127 - 0s - loss: 0.0129 - mse: 0.0129
Epoch 4/500
127/127 - 0s - loss: 0.0101 - mse: 0.0101
Epoch 5/500
127/127 - 0s - loss: 0.0115 - mse: 0.0115
Epoch 6/500
127/127 - 0s - loss: 0.0098 - mse: 0.0098
Epoch 7/500
127/127 - 0s - loss: 0.0093 - mse: 0.0093
Epoch 8/500
127/127 - 0s - loss: 0.0116 - mse: 0.0116
Epoch 9/500
127/127 - 0s - loss: 0.0117 - mse: 0.0117
Epoch 10/500
127/127 - 0s - loss: 0.0090 - mse: 0.0090
Epoch 11/500
127/127 - 0s - loss: 0.0085 - mse: 0.0085
Epoch 12/500
127/127 - 0s - loss: 0.0087 - mse: 0.0087
Epoch 13/500
127/127 - 0s - loss: 0.0084 - mse: 0.0084
Epoch 14/500
127/127 - 0s - loss: 0.0072 - mse: 0.0072
Epoch 15/500
127/127 - 0s - loss: 0.0101 - mse: 0.0101
Epoch 16/500
127/127 - 0s - loss: 0.0079 - mse: 0.0079
Epoch 17/500
127/127 - 0s - loss: 0.0071 - mse: 0.0071
Epoch 18/500
127/127 - 0s - loss: 0.0075 - mse: 0.007

Epoch 145/500
127/127 - 0s - loss: 3.8006e-04 - mse: 3.8006e-04
Epoch 146/500
127/127 - 0s - loss: 4.9481e-04 - mse: 4.9481e-04
Epoch 147/500
127/127 - 0s - loss: 8.2695e-04 - mse: 8.2695e-04
Epoch 148/500
127/127 - 0s - loss: 6.0599e-04 - mse: 6.0599e-04
Epoch 149/500
127/127 - 0s - loss: 5.4761e-04 - mse: 5.4761e-04
Epoch 150/500
127/127 - 0s - loss: 3.8913e-04 - mse: 3.8913e-04
Epoch 151/500
127/127 - 0s - loss: 2.7978e-04 - mse: 2.7978e-04
Epoch 152/500
127/127 - 0s - loss: 2.8353e-04 - mse: 2.8353e-04
Epoch 153/500
127/127 - 0s - loss: 2.0791e-04 - mse: 2.0791e-04
Epoch 154/500
127/127 - 0s - loss: 2.0825e-04 - mse: 2.0825e-04
Epoch 155/500
127/127 - 0s - loss: 2.0131e-04 - mse: 2.0131e-04
Epoch 156/500
127/127 - 0s - loss: 2.2347e-04 - mse: 2.2347e-04
Epoch 157/500
127/127 - 0s - loss: 2.3741e-04 - mse: 2.3741e-04
Epoch 158/500
127/127 - 0s - loss: 2.9954e-04 - mse: 2.9954e-04
Epoch 159/500
127/127 - 0s - loss: 2.5797e-04 - mse: 2.5797e-04
Epoch 160/500
127/127 - 0s - loss: 3.718

Epoch 277/500
127/127 - 0s - loss: 2.0611e-05 - mse: 2.0611e-05
Epoch 278/500
127/127 - 0s - loss: 2.4619e-05 - mse: 2.4619e-05
Epoch 279/500
127/127 - 0s - loss: 3.4794e-05 - mse: 3.4794e-05
Epoch 280/500
127/127 - 0s - loss: 2.8801e-05 - mse: 2.8801e-05
Epoch 281/500
127/127 - 0s - loss: 3.0601e-05 - mse: 3.0601e-05
Epoch 282/500
127/127 - 0s - loss: 2.6485e-05 - mse: 2.6485e-05
Epoch 283/500
127/127 - 0s - loss: 3.0487e-05 - mse: 3.0487e-05
Epoch 284/500
127/127 - 0s - loss: 3.9374e-05 - mse: 3.9374e-05
Epoch 285/500
127/127 - 0s - loss: 5.3518e-05 - mse: 5.3518e-05
Epoch 286/500
127/127 - 0s - loss: 4.2557e-05 - mse: 4.2557e-05
Epoch 287/500
127/127 - 0s - loss: 4.1931e-05 - mse: 4.1931e-05
Epoch 288/500
127/127 - 0s - loss: 6.4555e-05 - mse: 6.4555e-05
Epoch 289/500
127/127 - 0s - loss: 6.6780e-05 - mse: 6.6780e-05
Epoch 290/500
127/127 - 0s - loss: 7.6438e-05 - mse: 7.6438e-05
Epoch 291/500
127/127 - 0s - loss: 8.2356e-05 - mse: 8.2356e-05
Epoch 292/500
127/127 - 0s - loss: 7.167

Epoch 407/500
127/127 - 0s - loss: 2.2375e-04 - mse: 2.2375e-04
Epoch 408/500
127/127 - 0s - loss: 2.5950e-04 - mse: 2.5950e-04
Epoch 409/500
127/127 - 0s - loss: 2.0512e-04 - mse: 2.0512e-04
Epoch 410/500
127/127 - 0s - loss: 2.6798e-04 - mse: 2.6798e-04
Epoch 411/500
127/127 - 0s - loss: 6.9810e-04 - mse: 6.9810e-04
Epoch 412/500
127/127 - 0s - loss: 0.0011 - mse: 0.0011
Epoch 413/500
127/127 - 0s - loss: 0.0022 - mse: 0.0022
Epoch 414/500
127/127 - 0s - loss: 0.0013 - mse: 0.0013
Epoch 415/500
127/127 - 0s - loss: 6.7538e-04 - mse: 6.7538e-04
Epoch 416/500
127/127 - 0s - loss: 6.9040e-04 - mse: 6.9040e-04
Epoch 417/500
127/127 - 0s - loss: 0.0013 - mse: 0.0013
Epoch 418/500
127/127 - 0s - loss: 0.0013 - mse: 0.0013
Epoch 419/500
127/127 - 0s - loss: 0.0014 - mse: 0.0014
Epoch 420/500
127/127 - 0s - loss: 0.0012 - mse: 0.0012
Epoch 421/500
127/127 - 0s - loss: 8.7359e-04 - mse: 8.7359e-04
Epoch 422/500
127/127 - 0s - loss: 5.7173e-04 - mse: 5.7173e-04
Epoch 423/500
127/127 - 0s - los

In [15]:
import pickle
pickle.dump(lr, open('lin_model_capacitors_removed.pkl', 'wb'))
pickle.dump(scaler1, open('scaler_input_capacitors_removed.pkl', 'wb'))
pickle.dump(scaler2, open('scaler_output_capacitors_removed.pkl', 'wb'))